### Exercises — Solutions

#### Question 1 — Approx min/max over an interval

Approximate the minimum or maximum of a 1D function on `[start, end]` by sampling a given number of points (`resolution`).

In [20]:
from typing import Callable, Literal, Tuple
import math

Mode = Literal["min", "max"]

def approx_extreme(
    f: Callable[[float], float],
    start: float,
    end: float,
    *,
    resolution: int = 1_001,
    mode: Mode = "min",
    include_end: bool = True,
) -> Tuple[float, float]:
    """Return (x*, f(x*)) where x* approximately minimizes or maximizes f on [start, end].

    - Uniformly samples `resolution` points.
    - `mode` chooses min or max.
    - Swaps interval if start > end.
    - Single-pass selection (no NumPy required).
    """
    if resolution < 2:
        raise ValueError("resolution must be >= 2")
    if mode not in ("min", "max"):
        raise ValueError("mode must be 'min' or 'max'")
    a, b = (start, end) if start <= end else (end, start)
    n = resolution
    denom = (n - 1) if include_end else n
    step = (b - a) / denom if denom > 0 else 0.0
    xs = (a + i * step for i in range(n))
    if mode == "min":
        x_star, y_star = min(((x, f(x)) for x in xs), key=lambda t: t[1])
    else:
        x_star, y_star = max(((x, f(x)) for x in xs), key=lambda t: t[1])
    return x_star, y_star

# Demo
f = lambda x: x**2 - 1
x_min, y_min = approx_extreme(f, -5, 5, resolution=2001, mode="min")
x_max, y_max = approx_extreme(f, -5, 5, resolution=2001, mode="max")
print("Q1 demo min:", (x_min, y_min))
print("Q1 demo max:", (x_max, y_max))

Q1 demo min: (0.0, -1.0)
Q1 demo max: (-5.0, 24.0)


#### Question 2 — Apply function to many points and time approaches

Use a `for` loop, a list comprehension, and `map`, then time each with `timeit`.

In [21]:
import math
from timeit import timeit
from typing import Iterable, List, Tuple

Point = Tuple[float, float]

def func(point: Point) -> float:
    x, y = point
    return math.hypot(x, y)

points: List[Point] = [
    (0, 0),
    (1, 1),
    (10, 20),
    (math.pi, math.e),
]

expected_small = [0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313314]

# Larger dataset for timing
points_large: List[Point] = [(math.sin(x), math.cos(x)) for x in range(1, 1_000_000)]
len(points_large)

999999

In [24]:
def apply_for_loop(points: Iterable[Point]) -> List[float]:
    out: List[float] = []
    append = out.append  # micro-optimization
    for p in points:
        append(func(p))
    return out

def apply_list_comp(points: Iterable[Point]) -> List[float]:
    return [func(p) for p in points]

def apply_map(points: Iterable[Point]) -> List[float]:
    return list(map(func, points))

# Verify correctness on small sample
res_for = apply_for_loop(points)
res_list = apply_list_comp(points)
res_map = apply_map(points)
print("Q2 small (for): ", res_for)
print("Q2 small (list):", res_list)
print("Q2 small (map): ", res_map)

Q2 small (for):  [0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]
Q2 small (list): [0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]
Q2 small (map):  [0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]


In [25]:
REPEATS = 5  # Increase for tighter measurements

t_for = timeit(lambda: apply_for_loop(points_large), number=REPEATS)
t_list = timeit(lambda: apply_list_comp(points_large), number=REPEATS)
t_map = timeit(lambda: apply_map(points_large), number=REPEATS)

print(f"Q2 timing (repeats={REPEATS}):")
print(f"  for-loop:          {t_for:.3f} s")
print(f"  list comprehension: {t_list:.3f} s")
print(f"  map:                {t_map:.3f} s")

Q2 timing (repeats=5):
  for-loop:          1.203 s
  list comprehension: 1.126 s
  map:                1.171 s


#### Question 3 — Partial function (fix all args after the first)

Return a function that fixes all arguments **except** the first to given values.

In [26]:
from functools import update_wrapper
from typing import Any, Callable
import math

def partial(func: Callable[..., Any], *fixed_after_first: Any, **fixed_kw: Any) -> Callable[[Any], Any]:
    """Return g(x) that calls func(x, *fixed_after_first, **fixed_kw)."""
    def wrapper(first: Any) -> Any:
        return func(first, *fixed_after_first, **fixed_kw)
    return update_wrapper(wrapper, func)

# Example base functions
def power(x: float, n: float) -> float:
    return x ** n

def dist(pt1, pt2):
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(pt1, pt2)))

# Build specialized functions
squares = partial(power, 2)
dist_from_origin = partial(dist, (0, 0))
gcd_13 = partial(math.gcd, 13)
log_2 = partial(math.log, 2)
log_10 = partial(math.log, 10)
log_16 = partial(math.log, 16)

# Demos
print("Q3 squares(3):", squares(3))              # 9
print("Q3 squares(4):", squares(4))              # 16
print("Q3 dist_from_origin((1,1)):", round(dist_from_origin((1,1)), 3))
print("Q3 gcd_13(26):", gcd_13(26))              # 13
print("Q3 log_2(10):", round(log_2(10), 4))      # ~3.3219
print("Q3 log_10(10):", log_10(10))              # 1.0
print("Q3 log_16(10):", round(log_16(10), 4))    # ~0.8304

Q3 squares(3): 9
Q3 squares(4): 16
Q3 dist_from_origin((1,1)): 1.414
Q3 gcd_13(26): 13
Q3 log_2(10): 3.3219
Q3 log_10(10): 1.0
Q3 log_16(10): 0.8305


#### Question 4 — Logged execution wrapper

`logged(f)` returns a function that prints the execution time and returns the original result.

In [27]:
from time import perf_counter
from functools import wraps
from typing import TypeVar, Callable, Any

F = TypeVar("F", bound=Callable[..., Any])

def logged(f: F) -> F:
    @wraps(f)
    def wrapper(*args, **kwargs):  # type: ignore[misc]
        t0 = perf_counter()
        try:
            return f(*args, **kwargs)
        finally:
            dt_ms = (perf_counter() - t0) * 1_000.0
            print(f"[logged] {f.__name__} ran in {dt_ms:.3f} ms (args={len(args)}, kwargs={len(kwargs)})")
    return wrapper  # type: ignore[return-value]

# Example functions
def norm(x: float, y: float) -> float:
    return math.sqrt(x**2 + y**2)

def find_index_min(seq):
    m = min(seq)
    return seq.index(m)

# Logged variants + demos
norm_logged = logged(norm)
find_index_min_logged = logged(find_index_min)
print("Q4 norm_logged:", norm_logged(3, 4))
print("Q4 find_index_min_logged:", find_index_min_logged([3, 1, 2, 0, 5]))

[logged] norm ran in 0.007 ms (args=2, kwargs=0)
Q4 norm_logged: 5.0
[logged] find_index_min ran in 0.005 ms (args=1, kwargs=0)
Q4 find_index_min_logged: 3
